In [1]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('Sales Transaction v.4a.csv')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536350 entries, 0 to 536349
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionNo  536350 non-null  object 
 1   Date           536350 non-null  object 
 2   ProductNo      536350 non-null  object 
 3   ProductName    536350 non-null  object 
 4   Price          536350 non-null  float64
 5   Quantity       536350 non-null  int64  
 6   CustomerNo     536295 non-null  float64
 7   Country        536350 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 32.7+ MB


In [3]:
df['Date'] = pd.to_datetime(df['Date'])

In [4]:
df

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,581482,2019-12-09,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom
1,581475,2019-12-09,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom
2,581475,2019-12-09,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom
3,581475,2019-12-09,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069.0,United Kingdom
4,581475,2019-12-09,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069.0,United Kingdom
...,...,...,...,...,...,...,...,...
536345,C536548,2018-12-01,22168,Organiser Wood Antique White,18.96,-2,12472.0,Germany
536346,C536548,2018-12-01,21218,Red Spotty Biscuit Tin,14.09,-3,12472.0,Germany
536347,C536548,2018-12-01,20957,Porcelain Hanging Bell Small,11.74,-1,12472.0,Germany
536348,C536548,2018-12-01,22580,Advent Calendar Gingham Sack,16.35,-4,12472.0,Germany


In [5]:
product_sell = df.groupby('ProductName')['Quantity'].sum().reset_index()

In [6]:
df['TotalAmount'] = df['Price']*df['Quantity']

In [7]:
import plotly.express as px
product_sell_top = product_sell[product_sell['Quantity']>20000]
fig = px.bar(product_sell_top, x=product_sell_top['ProductName'], y=product_sell_top['Quantity'], text_auto='position', title='Products with Total Sell Over 20k Quantities')
fig.update_layout(xaxis_title='Product Name', yaxis_title='Total Sell')
fig.show()

In [8]:
product_revenue = df.groupby('ProductName')['TotalAmount'].sum().reset_index()

In [9]:
top_total_product_revenue = product_revenue[product_revenue['TotalAmount']>200000].round(2)
fig = px.bar(top_total_product_revenue, x=top_total_product_revenue['ProductName'], y=top_total_product_revenue['TotalAmount'], text_auto='position', title='Products with Total Revenue Over 200k')
fig.update_layout(xaxis_title='Product Name', yaxis_title='Total Revenue')
fig.show()

In [10]:
customer_expanse = df.groupby('CustomerNo')['TotalAmount'].sum().reset_index()
customer_expanse['CustomerNo'] = customer_expanse['CustomerNo'].astype(str)

In [11]:
top_total_customer_expanse = customer_expanse[customer_expanse['TotalAmount']>500000].round(2)

fig = px.bar(top_total_customer_expanse, x=top_total_customer_expanse['CustomerNo'], y=top_total_customer_expanse['TotalAmount'], text_auto='position')
fig.update_layout(xaxis_title='Customer No.', yaxis_title='Total Amount', title='Top Buyers(Spent More than 500k)')
fig.show()

In [12]:
customer_by_country = df.groupby('Country')['CustomerNo'].count().reset_index().sort_values(by=['CustomerNo'], ascending=False)
print('Hightest Number of Customer by Country :\n',customer_by_country.head())

Hightest Number of Customer by Country :
            Country  CustomerNo
36  United Kingdom      485040
14         Germany       10675
13          France       10526
10            EIRE        8048
3          Belgium        2539


In [13]:
product_sell_by_country =  df.groupby(['Country'])['Quantity'].sum().reset_index().sort_values(by=['Quantity'], ascending=False)
print('Hightest Total Product Sell by Country :\n',product_sell_by_country.head())

Hightest Total Product Sell by Country :
            Country  Quantity
36  United Kingdom   4409751
24     Netherlands    200510
10            EIRE    142549
14         Germany    118630
13          France    117010


In [14]:
revenue_by_country = df.groupby('Country')['TotalAmount'].sum().reset_index().sort_values(by=['TotalAmount'], ascending=False)
print('Hightest Total Product Revenue by Country :\n',revenue_by_country.head())

Hightest Total Product Revenue by Country :
            Country  TotalAmount
36  United Kingdom  49994030.17
24     Netherlands   2147811.39
10            EIRE   1660645.07
14         Germany   1350265.40
13          France   1316880.98


In [15]:
top_customer_by_country = df.groupby(['Country','CustomerNo'])['TotalAmount'].sum().reset_index().sort_values(by=['TotalAmount'], ascending=False)
print('Top Customers :\n',top_customer_by_country.head())

Top Customers :
              Country  CustomerNo  TotalAmount
315      Netherlands     14646.0   2108959.95
4592  United Kingdom     18102.0    897137.36
3          Australia     12415.0    895267.24
4095  United Kingdom     17450.0    876816.01
76              EIRE     14911.0    873037.90


In [16]:
product_revenue_overall = df.groupby(['Date','Country'])['TotalAmount'].sum().reset_index()
product_revenue_uk = product_revenue_overall[product_revenue_overall['Country']=='United Kingdom']
product_revenue_other_countries = product_revenue_overall[product_revenue_overall['Country'] !='United Kingdom']
product_revenue_other_countries = product_revenue_other_countries.groupby('Date')['TotalAmount'].sum().reset_index()

In [17]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=product_revenue_uk['Date'], y=product_revenue_uk['TotalAmount'], name='United Kingdom'))
fig.add_trace(go.Scatter(x=product_revenue_other_countries['Date'], y=product_revenue_other_countries['TotalAmount'], name='Other Countries'))
fig.update_layout(xaxis_title='Date', yaxis_title='Net Amount', title='Product Revenue in United Kingdom vs Other Countries Over the Time')
fig.show()

In [18]:
product_sell_overall = df.groupby(['Date','Country'])['Quantity'].sum().reset_index()
product_sell_uk = product_sell_overall[product_sell_overall['Country']=='United Kingdom']
product_sell_other_countries = product_sell_overall[product_sell_overall['Country'] !='United Kingdom']
product_sell_other_countries = product_sell_other_countries.groupby('Date')['Quantity'].sum().reset_index()

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=product_sell_uk['Date'], y=product_sell_uk['Quantity'], name='United Kingdom'))
fig.add_trace(go.Scatter(x=product_sell_other_countries['Date'], y=product_sell_other_countries['Quantity'], name='Other Countries'))
fig.update_layout(xaxis_title='Date',yaxis_title='Net Quantity', title='Product Sell in United Kingdom vs Other Countries Over the Time')
fig.show()

In [20]:
customer_overall = df.groupby(['Date','Country'])['CustomerNo'].count().reset_index()
customer_uk = customer_overall[customer_overall['Country']=='United Kingdom']
customer_other_countries = customer_overall[customer_overall['Country'] !='United Kingdom']
customer_other_countries = customer_other_countries.groupby('Date')['CustomerNo'].sum().reset_index()

In [21]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=customer_uk['Date'], y=customer_uk['CustomerNo'], name='United Kingdom'))
fig.add_trace(go.Scatter(x=customer_other_countries['Date'], y=customer_other_countries['CustomerNo'], name='Other Countries'))
fig.update_layout(xaxis_title='Date',yaxis_title='Total Customer', title='Total Number of Customer in United Kingdom vs Other Countries Over the Time')
fig.show()

In [22]:
total_loss = df[df['TotalAmount']<0]
total_profit = df[df['TotalAmount']>0]

In [23]:
total_loss.sort_values(by=['TotalAmount'], ascending=True)
total_loss_by_date = total_loss.groupby('Date')['TotalAmount'].sum().reset_index()
total_loss_by_date.rename(columns={'TotalAmount':'TotalAmountt'}, inplace=True)

In [24]:
total_profit.sort_values(by=['TotalAmount'], ascending=False)
total_profit_by_date = total_profit.groupby('Date')['TotalAmount'].sum().reset_index()

In [25]:
total_revenue_by_date = total_profit_by_date.merge(total_loss_by_date, on=['Date'], how='outer')
total_revenue_by_date['TotalAmount'] = total_revenue_by_date['TotalAmount']+total_revenue_by_date['TotalAmountt']
total_revenue_by_date = total_revenue_by_date.drop(columns='TotalAmountt', axis='columns')

In [26]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=total_profit_by_date['Date'], y=total_profit_by_date['TotalAmount'], name='Total Profit'))
fig.add_trace(go.Scatter(x=total_loss_by_date['Date'], y=total_loss_by_date['TotalAmountt'], name='Total Loss'))
fig.add_trace(go.Scatter(x=total_revenue_by_date['Date'], y=total_revenue_by_date['TotalAmount'], name='Total Revenue'))
fig.update_layout(xaxis_title='Date',yaxis_title='Total Profit', title='Total Revenue vs Total Profit vs Total Loss(Loss Refer the Return Products) Over the Time')
fig.show()

In [27]:
total_loss_by_country = total_loss.groupby('Country')['TotalAmount'].sum().reset_index()
total_loss_by_country = total_loss_by_country.sort_values(by=['TotalAmount'], ascending=True)
total_loss_by_country.head()

,Country,TotalAmount
24,United Kingdom,-2530628.30
7,EIRE,-52765.88
11,Germany,-21277.87
23,USA,-15418.27
20,Spain,-15273.47


In [28]:
total_profit_by_country = total_profit.groupby('Country')['TotalAmount'].sum().reset_index()
total_profit_by_country = total_profit_by_country.sort_values(by=['TotalAmount'], ascending=False)
total_profit_by_country.head()

,Country,TotalAmount
36,United Kingdom,52524658.47
24,Netherlands,2151553.59
10,EIRE,1713410.95
14,Germany,1371543.27
13,France,1330652.89


In [29]:
other_countries_total_profit = total_profit_by_country[total_profit_by_country['Country']!='United Kingdom']['TotalAmount'].sum()
uk_total_profit = total_profit_by_country[total_profit_by_country['Country']=='United Kingdom']
other_countries_total_loss = total_loss_by_country[total_loss_by_country['Country']!='United Kingdom']['TotalAmount'].sum()
uk_total_loss = total_loss_by_country[total_loss_by_country['Country']=='United Kingdom']

In [30]:
uk_revenue = uk_total_profit.merge(uk_total_loss, on='Country')
uk_revenue['Revenue'] = uk_revenue['TotalAmount_x']+uk_revenue['TotalAmount_y']

In [31]:
import numpy as np
other_countries_total_profitt = pd.DataFrame(np.array([['Other Countries', other_countries_total_profit]]),columns=['Country', 'TotalAmount'])
other_countries_total_losss = pd.DataFrame(np.array([['Other Countries', other_countries_total_loss]]),columns=['Country', 'TotalAmount'])
other_countries_revenue = other_countries_total_profitt.merge(other_countries_total_losss, on='Country')
other_countries_revenue['Revenue'] = (other_countries_revenue['TotalAmount_x']).astype(float)+(other_countries_revenue['TotalAmount_y']).astype(float)
other_countries_revenue['TotalAmount_x'] = other_countries_revenue['TotalAmount_x'].astype('float')
other_countries_revenue['TotalAmount_y'] = other_countries_revenue['TotalAmount_y'].astype(float)

In [32]:
total_revenue = pd.concat([uk_revenue,other_countries_revenue],axis=0)

In [33]:
fig = go.Figure()

fig.add_trace(go.Bar(x=total_revenue['Country'], y=total_revenue['TotalAmount_x'], name='Total Profit'))
fig.add_trace(go.Bar(x=total_revenue['Country'], y=total_revenue['TotalAmount_y'], name='Total Loss'))
fig.add_trace(go.Bar(x=total_revenue['Country'], y=total_revenue['Revenue'], name='Total Revenue'))
fig.update_layout(xaxis_title='Country', yaxis_title='Total Amount', title='Profit, Loss, and Revenue by United Kingdom vs Other Countries')
fig.show()
    

In [34]:
total_profit_by_date = total_profit.groupby(['Date','Country'])['TotalAmount'].sum().reset_index()
uk_profit_by_date = total_profit_by_date[total_profit_by_date['Country']=='United Kingdom']
total_loss_by_date = total_loss.groupby(['Date','Country'])['TotalAmount'].sum().reset_index()
uk_loss_by_date = total_loss_by_date[total_loss_by_date['Country']=='United Kingdom']

In [35]:
uk_revenue_by_date = uk_profit_by_date.merge(uk_loss_by_date, on=['Date'], how='outer')
uk_revenue_by_date['TotalAmount'] = uk_revenue_by_date['TotalAmount_x']+uk_revenue_by_date['TotalAmount_y']

In [36]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=uk_profit_by_date['Date'], y=uk_profit_by_date['TotalAmount'], name='Profit'))
fig.add_trace(go.Scatter(x=uk_loss_by_date['Date'], y=uk_loss_by_date['TotalAmount'], name='Loss'))
fig.add_trace(go.Scatter(x=uk_revenue_by_date['Date'], y=uk_revenue_by_date['TotalAmount'], name='Reveue'))
fig.update_layout(xaxis_title='Date', yaxis_title='Total Amount',title='Revenue vs Profit vs Loss in United Kingdom by Date')

fig.show()

In [119]:
product_revenue_by_country = df.groupby(['Country','ProductName'])['TotalAmount'].sum().reset_index()

top_product_revenue_by_country = pd.DataFrame(np.array(['Antique Silver T-Light Glass', 'Assorted Colour Bird Ornament', 'Assorted Colours Silk Fan', 'Brocade Ring Purse', 'Charlotte Bag Suki Design', 'Cream Hanging Heart T-Light Holder', 'Heart Of Wicker Small',
            'Jumbo Bag Red Retrospot','Mini Paint Set Vintage','Pack Of 12 London Tissues','Pack Of 60 Pink Paisley Cake Cases','Pack Of 72 Retrospot Cake Case',"Paper Chain Kit 50'S Christmas",'Paper Craft Little Birdie','Party Bunting','Popcorn Holder','Rabbit Night Light','Red Harmonica In Box',
            'Regency Cakestand 3 Tier'	,'Victorian Glass Hanging T-Light' ,'World War 2 Gliders Asstd Designs']), columns=['ProductName'])

top_product_revenue_by_country = top_product_revenue_by_country.merge(product_revenue_by_country, on='ProductName', how='inner')

In [123]:
top_product_revenue_by_country.groupby(['Country','ProductName'])['TotalAmount'].sum().reset_index()

,Country,ProductName,TotalAmount
0,Australia,Assorted Colours Silk Fan,6594.00
1,Australia,Cream Hanging Heart T-Light Holder,79.62
2,Australia,Heart Of Wicker Small,225.00
3,Australia,Jumbo Bag Red Retrospot,1305.50
4,Australia,Mini Paint Set Vintage,31525.92
...,...,...,...
311,Unspecified,Rabbit Night Light,12.38
312,Unspecified,Red Harmonica In Box,138.36
313,Unspecified,Regency Cakestand 3 Tier,46.64
314,Unspecified,Victorian Glass Hanging T-Light,138.36


In [126]:
fig = px.treemap(top_product_revenue_by_country, path=['Country','ProductName'], values='TotalAmount', color='ProductName', color_continuous_scale='RdBu',)
fig.show()

In [ ]:
total_loss_by_customer = total_loss.groupby('CustomerNo')['TotalAmount'].sum().reset_index()
total_loss_by_customer.sort_values(by=['TotalAmount'], ascending=True)

,CustomerNo,TotalAmount
0,12346.0,-840113.80
1093,16446.0,-501359.05
927,15749.0,-115955.30
958,15838.0,-96220.80
997,16029.0,-92288.44
...,...,...
1176,16759.0,-6.13
590,14477.0,-6.04
693,14869.0,-6.04
4,12375.0,-6.04


In [ ]:
total_profit_by_customer = total_profit.groupby('CustomerNo')['TotalAmount'].sum().reset_index()
total_profit_by_customer.sort_values(by=['TotalAmount'], ascending=False)

,CustomerNo,TotalAmount
1880,14646.0,2112282.03
3302,16446.0,1002741.57
2085,14911.0,914633.52
126,12415.0,900545.54
4581,18102.0,897137.36
...,...,...
61,12309.0,12.86
1716,14435.0,12.38
453,12810.0,11.98
1208,13775.0,11.53


In [ ]:
total_loss_by_product = total_loss.groupby('ProductName')['TotalAmount'].sum().reset_index()
total_loss_by_product.sort_values(by=['ProductName'], ascending=True)

,ProductName,TotalAmount
0,10 Colour Spaceboy Pen,-1834.80
1,12 Coloured Party Balloons,-218.40
2,12 Egg House Painted Wood,-23.32
3,12 Ivory Rose Peg Place Settings,-11.53
4,12 Pencil Small Tube Woodland,-43.68
...,...,...
1908,Zinc Sweetheart Wire Letter Rack,-56.36
1909,Zinc T-Light Holder Star Large,-123.42
1910,Zinc T-Light Holder Stars Small,-432.74
1911,Zinc Top 2 Door Wooden Shelf,-129.90


In [ ]:
total_profit_by_product = total_profit.groupby('ProductName')['TotalAmount'].sum().reset_index()
total_profit_by_product.sort_values(by=['ProductName'], ascending=True)

,ProductName,TotalAmount
0,10 Colour Spaceboy Pen,71972.51
1,12 Coloured Party Balloons,22933.78
2,12 Daisy Pegs In Wood Box,4192.68
3,12 Egg House Painted Wood,3787.54
4,12 Hanging Eggs Hand Painted,366.47
...,...,...
3748,Zinc T-Light Holder Stars Small,55344.28
3749,Zinc Top 2 Door Wooden Shelf,410.04
3750,Zinc Willie Winkie Candle Stick,31512.83
3751,Zinc Wire Kitchen Organiser,414.20
